In [6]:
# Props to this sensei
# https://www.youtube.com/watch?v=kCc8FmEb1nY&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=8

In [ ]:
import pytorch as pt

In [5]:
# read file
text_file = "tiny-shakespeare.txt"
with open(text_file, "r") as f:
    text = f.read()
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [10]:
chars = list(set(text))
chars

['u',
 'l',
 'P',
 'c',
 'T',
 'j',
 'w',
 'h',
 'f',
 ',',
 '.',
 'S',
 'm',
 'r',
 'v',
 'O',
 'g',
 's',
 'V',
 'x',
 'J',
 'K',
 ':',
 'Z',
 '$',
 '\n',
 'p',
 'a',
 'M',
 'd',
 '-',
 'b',
 'G',
 't',
 'H',
 'Y',
 'i',
 'B',
 'Q',
 'k',
 'L',
 'R',
 '?',
 'C',
 'z',
 'A',
 'I',
 ';',
 'W',
 "'",
 'D',
 '&',
 '!',
 'n',
 'X',
 '3',
 'o',
 'E',
 'e',
 'U',
 ' ',
 'q',
 'y',
 'N',
 'F']